# Imports

In [23]:
import pandas as pd
import numpy as np
import os
from sklearn import metrics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
# this adds the src folder in the sys path, where the metric_utils.py file is
# not needed if this notebook is in the same folder, but uncomment to access from the data subfolders
sys.path.append( '..' )
from metric_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Result path

In [24]:
dataset_name = 'ucberkeley hate speech'
model_name = 'bert'

# change to ../../results/{dataset_name} when using inside one of the data subfolders
result_folder = f'../../results/{dataset_name}'
test_csv_filepath = os.path.join(result_folder, 'test.csv')

model_folder = os.path.join(result_folder, model_name) # for this particular model
result_filepath = os.path.join(model_folder, 'results.csv')
dp_result_filepath = os.path.join(model_folder, 'results_dp.csv')

# Dataset
The dataset can be preprocessed from the original dataset to be used here. To simplify things, I saved the preprocessed datasets during the tokenizing process as csv files and then downloaded them in the corresponding dataset folder of [`result`](../results/) directory.

You can recreate the processed datasets using the tokenize notebooks for that particular dataset. That would give you train, test and validation csv files as well as the tokenized data.

In [8]:
result = pd.read_csv(result_filepath)
dp_result = pd.read_csv(dp_result_filepath)
df = pd.read_csv(test_csv_filepath)

In [9]:
df.head(2)

,comment_id,text,target_gender_men,target_gender_women,target_race_white,target_race_black,labels
0,41012,I don't want to see America treated like a garbage can. Luv from Canada.,0,0,0,0,0
1,31056,@mike_pence @realDonaldTrump You're a Christian of convenience. You're a fake,0,0,0,0,0


## Split test result

In [10]:
# only calculate test result
result = result[result['split']=='test']
dp_result = dp_result[dp_result['split']=='test']

# drop split column
result.drop(columns=['split'], inplace=True)
dp_result.drop(columns=['split'], inplace=True)

## Prepare test data

In [11]:
# result has id column which is the same as the text ids from raw dataset
raw_id_column = 'comment_id'
df.rename({raw_id_column: id_column}, axis=1, inplace=True)
df.drop(columns=['labels'], inplace=True)

## Merge

In [12]:
result = result.merge(df, on=id_column, how='inner').reset_index(drop=True)
dp_result = dp_result.merge(df, on=id_column, how='inner').reset_index(drop=True)

In [13]:
result.head()

,id,labels,probs,text,target_gender_men,target_gender_women,target_race_white,target_race_black
0,41012,0,0.158559,I don't want to see America treated like a garbage can. Luv from Canada.,0,0,0,0
1,31056,0,0.281979,@mike_pence @realDonaldTrump You're a Christian of convenience. You're a fake,0,0,0,0
2,17058,0,0.405848,Jesus just has his holy reverse card that he uses on the microbes. Does that girl have a holy reverse? I thot not.,0,0,0,0
3,4023,0,0.062836,"Why? It's never even hinted that being gay or bi means you end up in the Bad Place, if that's what you meant?",0,0,0,0
4,3792,0,0.754239,I'm half Jewish you fucking idiot. Eat a dick,0,0,0,0


# Evaluation

## Convert probability to prediction

In [15]:
result[prediction_column] = result[probability_column] >=0.5
dp_result[prediction_column] = dp_result[probability_column] >=0.5

## Identity groups

In [16]:
group_map = {
    'gender': {
        'unprivileged':['target_gender_women'],
        'privileged':['target_gender_men']
    },
    'race': {
        'unprivileged':['target_race_black'],
        'privileged': ['target_race_white']
    }
}

identities = []
for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    for subgroup_key in subgroup_map.keys():
        identities.extend(subgroup_map[subgroup_key])

print(identities)

['target_gender_women', 'target_gender_men', 'target_race_black', 'target_race_white']


### Binarize identity and target columns

In [18]:
result = binarize(result, [target_column] + identities)
dp_result = binarize(dp_result, [target_column] + identities)

## Bias

In [19]:
bias_results = {
   'fairness_metrics': ['demographic parity', 'Equality of Opportunity (w.r.t y = 1)',
   'Equality of Opportunity (w.r.t y = 0)', 'Equality of Odds', 'unprotected-accuracy',
   'protected-accuracy', 'accuracy']
}

for group_key in group_map.keys():
   subgroup_map = group_map[group_key]
   privileged_group = subgroup_map['privileged']
   unprivileged_group = subgroup_map['unprivileged']

   bias_results[group_key] = calculate_bias(result, privileged_group, unprivileged_group)
   bias_results[group_key+'_DP'] = calculate_bias(dp_result, privileged_group, unprivileged_group)

bias_results = pd.DataFrame(bias_results) 
bias_results

,fairness_metrics,gender,gender_DP,race,race_DP
0,demographic parity,0.472754,0.593520,0.541825,0.500000
1,Equality of Opportunity (w.r.t y = 1),0.948202,0.982615,0.732689,0.565546
2,Equality of Opportunity (w.r.t y = 0),0.957949,0.946227,0.963286,0.907790
3,Equality of Odds,0.953075,0.964421,0.847988,0.736668
4,unprotected-accuracy,0.758911,0.740117,0.801462,0.799026
5,protected-accuracy,0.784978,0.798233,0.756654,0.775665
6,accuracy,0.771945,0.769175,0.779058,0.787345


In [20]:
bias_results.to_csv(os.path.join(model_folder, 'bias.csv'), index=False)

## Overall metrics

In [21]:
overall_results = {
    'metrics': ['auc', 'accuracy', 'f1_score', 'precision', 'recall', 'true positive rate', 'false positive rate']
}

for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    privileged_group = subgroup_map['privileged']
    unprivileged_group = subgroup_map['unprivileged']

    privileged_group_name = ','.join(privileged_group)
    unprivileged_group_name = ','.join(unprivileged_group)

    overall_results[privileged_group_name] = calculate_metrics(result, privileged_group)
    overall_results[privileged_group_name + '_DP'] = calculate_metrics(dp_result, privileged_group)

    overall_results[unprivileged_group_name] = calculate_metrics(result, unprivileged_group)
    overall_results[unprivileged_group_name + '_DP'] = calculate_metrics(dp_result, unprivileged_group)

overall_results['Total'] = calculate_metrics(result, [])
overall_results['Total_DP'] = calculate_metrics(dp_result, [])

overall_results = pd.DataFrame(overall_results) 
overall_results

,metrics,target_gender_men,target_gender_men_DP,target_gender_women,target_gender_women_DP,target_race_white,target_race_white_DP,target_race_black,target_race_black_DP,Total,Total_DP
0,auc,0.821865,0.807471,0.813587,0.777163,0.787519,0.757466,0.872563,0.860212,0.850017,0.822671
1,accuracy,0.784978,0.798233,0.758911,0.740117,0.756654,0.775665,0.801462,0.799026,0.797061,0.797926
2,f1_score,0.601093,0.559486,0.646388,0.561749,0.536232,0.404040,0.777018,0.755556,0.642944,0.578435
3,precision,0.582011,0.649254,0.621572,0.626829,0.528571,0.645161,0.745407,0.784615,0.628419,0.687232
4,recall,0.621469,0.491525,0.673267,0.508911,0.544118,0.294118,0.811429,0.728571,0.658157,0.499377
5,true positive rate,0.621469,0.491525,0.673267,0.508911,0.544118,0.294118,0.811429,0.728571,0.658157,0.499377
6,false positive rate,0.157371,0.093625,0.199422,0.147399,0.169231,0.056410,0.205945,0.148620,0.149557,0.087341


In [22]:
overall_results.to_csv(os.path.join(model_folder, 'overall_results.csv'), index=False)